In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import numpy as np

In [3]:
target_value = '절도'
model_name = 'linear'

### Load Data

In [4]:
year = 2014

In [5]:
df1 = pd.read_csv('/content/drive/MyDrive/산업인공지능 대회/merged_data/'+str(year)+'.csv', encoding='utf-8')

In [6]:
df2 = pd.read_csv('/content/drive/MyDrive/산업인공지능 대회/merged_data/'+str(year+1)+'.csv', encoding='utf-8')

In [7]:
df3 = pd.read_csv('/content/drive/MyDrive/산업인공지능 대회/merged_data/'+str(year+2)+'.csv', encoding='utf-8')

In [8]:
df = pd.concat([df1,df2])

In [9]:
df = pd.concat([df,df3])

In [10]:
df = df.reset_index(drop=True)

In [11]:
df

,위치,장소,요일,시간대,인구수,절도,살인,강도,성폭력,폭행
0,서울,아파트ㆍ연립ㆍ다세대,일,00:00-02:59,10103233,0.79,1.46,1.67,1.52,1.69
1,서울,아파트ㆍ연립ㆍ다세대,일,03:00-05:59,10103233,0.78,1.50,1.90,1.59,1.53
2,서울,아파트ㆍ연립ㆍ다세대,일,06:00-08:59,10103233,0.63,1.27,0.90,1.42,1.22
3,서울,아파트ㆍ연립ㆍ다세대,일,09:00-11:59,10103233,1.13,1.59,0.93,1.38,1.27
4,서울,아파트ㆍ연립ㆍ다세대,일,12:00-14:59,10103233,1.21,1.56,1.06,1.44,1.28
...,...,...,...,...,...,...,...,...,...,...
150523,도시이외,공장,토,09:00-11:59,3750206,1.65,0.81,0.06,1.07,1.10
150524,도시이외,공장,토,12:00-14:59,3750206,1.87,0.84,0.30,1.15,1.11
150525,도시이외,공장,토,15:00-17:59,3750206,2.01,1.00,0.13,1.38,1.20
150526,도시이외,공장,토,18:00-20:59,3750206,2.07,0.90,0.54,1.66,1.46


### Preprocess Data

In [12]:
from sklearn.preprocessing import LabelEncoder

In [13]:
region = LabelEncoder()
region = region.fit(df['위치'])
df['위치'] = region.transform(df['위치'])

place = LabelEncoder()
place = place.fit(df['장소'])
df['장소'] = place.transform(df['장소'])

day = LabelEncoder()
day = day.fit(df['요일'])
df['요일'] = day.transform(df['요일'])

time = LabelEncoder()
time = time.fit(df['시간대'])
df['시간대'] = time.transform(df['시간대'])

In [21]:
import numpy
numpy.save('/content/drive/MyDrive/산업인공지능 대회/model/region.npy', region.classes_)
numpy.save('/content/drive/MyDrive/산업인공지능 대회/model/place.npy', place.classes_)
numpy.save('/content/drive/MyDrive/산업인공지능 대회/model/day.npy', day.classes_)
numpy.save('/content/drive/MyDrive/산업인공지능 대회/model/time.npy', time.classes_)

In [14]:
df

,위치,장소,요일,시간대,인구수,절도,살인,강도,성폭력,폭행
0,57,5,4,0,10103233,0.79,1.46,1.67,1.52,1.69
1,57,5,4,1,10103233,0.78,1.50,1.90,1.59,1.53
2,57,5,4,2,10103233,0.63,1.27,0.90,1.42,1.22
3,57,5,4,3,10103233,1.13,1.59,0.93,1.38,1.27
4,57,5,4,4,10103233,1.21,1.56,1.06,1.44,1.28
...,...,...,...,...,...,...,...,...,...,...
150523,36,0,5,3,3750206,1.65,0.81,0.06,1.07,1.10
150524,36,0,5,4,3750206,1.87,0.84,0.30,1.15,1.11
150525,36,0,5,5,3750206,2.01,1.00,0.13,1.38,1.20
150526,36,0,5,6,3750206,2.07,0.90,0.54,1.66,1.46


### Linear Model

In [15]:
from sklearn.linear_model import LinearRegression

In [16]:
def z_transform(X, mu, std):
    """
    ==========================
    | X: df, mu: df, std: df |
    ==========================
    """
    return (X-mu)/(std+0.000001)

def inv_z_transform(y, mu, std):
    """
    ====================================
    | y: ndarray, mu: df, std: df      |
    ====================================
    """
    return y * (std+0.000001) + mu

In [ ]:
x_td = df.drop(['절도', '살인',	'강도',	'성폭력',	'폭행'], axis=1)
y_td = df[target_value]

x_td_mu, x_td_std = x_td.mean(), x_td.std()
y_td_mu, y_td_std = y_td.mean(), y_td.std()

X = z_transform(x_td, x_td_mu, x_td_std)
y = z_transform(y_td, y_td_mu, y_td_std)

coeffs = {}
model = LinearRegression().fit(X=X, y=y)


for i, target in enumerate(X):
    coeffs[target] = model.coef_[i]


coeffs['const'] = model.intercept_

# linear skeleton model prediction
const = coeffs['const']

In [31]:
numpy.save('/content/drive/MyDrive/산업인공지능 대회/model/x_y_mu_std.npy',[x_td_mu,x_td_std,y_td_mu,y_td_std])

/usr/local/lib/python3.10/dist-packages/numpy/lib/npyio.py:518: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  arr = np.asanyarray(arr)


In [18]:
coeffs

{'위치': -0.032972119470592455,
 '장소': -0.12400905039989735,
 '요일': -0.004739199378340046,
 '시간대': 0.5031276086821369,
 '인구수': -0.021823790248067312,
 'const': 2.9997700910576625e-16}

In [19]:
pred_train = np.zeros(X.shape[0]) + const
be = 1
for key in coeffs.keys():
    if key =='const': continue
    #print ('{}: {:.4f}'.format(key, coeffs[key]))
    pred_train += (coeffs[key] / be) * X[key]

y = inv_z_transform(pred_train, y_td_mu, y_td_std)
error = df[target_value].values - y

print('RMSE : ', np.sqrt((error**2).mean()))
print('MAE : ', np.mean(np.abs(error)))

RMSE :  0.5150939593628726
MAE :  0.4188271064738904


In [29]:
y = inv_z_transform(model.predict(X), y_td_mu, y_td_std)

In [23]:
y_td

0         0.79
1         0.78
2         0.63
3         1.13
4         1.21
          ... 
150523    1.65
150524    1.87
150525    2.01
150526    2.07
150527    1.79
Name: 절도, Length: 150528, dtype: float64

### 모델 pickle

In [20]:
import pickle
pickle.dump(model, open('/content/drive/MyDrive/산업인공지능 대회/model/'+target_value+model_name+'.pickle', 'wb'))

### 다중 output

In [39]:
x_td = df.drop(['절도', '살인',	'강도',	'성폭력',	'폭행'], axis=1)
y_td = df[['절도', '살인',	'강도',	'성폭력',	'폭행']]

x_td_mu, x_td_std = x_td.mean(), x_td.std()
y_td_mu, y_td_std = y_td.mean(), y_td.std()

X = z_transform(x_td, x_td_mu, x_td_std)
y = z_transform(y_td, y_td_mu, y_td_std)

coeffs = {}
model = LinearRegression().fit(X=X, y=y)


for i, target in enumerate(X):
    coeffs[target] = model.coef_[i]


coeffs['const'] = model.intercept_

# linear skeleton model prediction
const = coeffs['const']

In [36]:
numpy.save('/content/drive/MyDrive/산업인공지능 대회/model/new_x_y_mu_std.npy',[x_td_mu,x_td_std,y_td_mu,y_td_std])

In [40]:
coeffs

{'위치': array([-0.03297212, -0.12400905, -0.0047392 ,  0.50312761, -0.02182379]),
 '장소': array([ 0.01702481, -0.02752231, -0.01301731,  0.42893203, -0.00990801]),
 '요일': array([-0.0139391 , -0.12254978, -0.15563822, -0.18626188, -0.01661724]),
 '시간대': array([-0.00096873,  0.06983569,  0.1471095 ,  0.40334803, -0.00931365]),
 '인구수': array([ 0.02308025, -0.15122994,  0.18410601,  0.22425935, -0.01479176]),
 'const': array([ 2.99977009e-16, -5.75729543e-17,  2.77271900e-16,  6.30147255e-16,
         1.32474641e-16])}

In [60]:
model.predict(X)

array([[-1.08268873, -0.78011033, -0.06078975, -0.55517236, -0.53699809],
       [-0.86310666, -0.59290975, -0.14208079, -0.37913751, -0.43912366],
       [-0.6435246 , -0.40570918, -0.22337183, -0.20310266, -0.34124922],
       ...,
       [ 0.47078828,  0.26715913, -0.1324106 ,  0.27964603,  0.49693524],
       [ 0.69037034,  0.45435971, -0.21370165,  0.45568088,  0.59480967],
       [ 0.90995241,  0.64156029, -0.29499269,  0.63171573,  0.69268411]])

In [45]:
y_td_mu

절도     1.301358
살인     1.516797
강도     1.355952
성폭력    1.371391
폭행     1.185986
dtype: float64

In [62]:
prediction = inv_z_transform(model.predict(X)[0], y_td_mu, y_td_std)

In [63]:
prediction

절도     0.648630
살인     1.017389
강도     1.320521
성폭력    1.051510
폭행     0.874861
dtype: float64

In [35]:
import pickle
pickle.dump(model, open('/content/drive/MyDrive/산업인공지능 대회/model/linear.pickle', 'wb'))